# Function Calling/Tool Use


## Just a quick definition

Function calling in the context of Large Language Models (LLMs) refers to the ability of the model to invoke external functions or tools during its response generation. Instead of only generating text, the LLM can recognize when a task requires external data or computation, call a predefined function with the appropriate arguments, and then use the function's output to continue its response.

Think of having a super smart robot that you can only speak with. Pretty useless at doing anything aside from talking. Now imagine you have given that robot a hammer and some nails. It can now put up that wall painting that's been waiting forever to be hung :D

Except, the way we as AI Engineers give AI tools is by defining python functions and describing that function in detail to the LLM so it knows what tools it has access to, what each tool can do, what are its inputs and expected outputs.

Just remember, a *tool* in its simplest form is just a *python function* that you have defined. It can be as simple as a calculator function or something like being able to call external APIs.

![tool-use](../images/tool-use.png)

*Image courtesy of [API Deck](https://www.apideck.com/blog/llm-tool-use-and-function-calling)*

## Step 1: Import libraries

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY is None:
    raise Exception("API key is missing")

## Step 2: Define a tool

In [2]:
import yfinance as yf

def get_stock_price(ticker: str):
    """
    Returns the latest available market price for the given stock ticker.
    """
    stock = yf.Ticker(ticker)
    data = stock.history(period="1d", interval="1m")
    
    if data.empty:
        raise Exception("No price data available.")
    
    latest_price = data["Close"].iloc[-1]
    return float(latest_price)

## Step 3: Call a chat model normally

In [3]:
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1-nano",
    input=[
        {"role": "user", "content": "What is the real-time stock price of Apple?"}
    ]
)

print(response.output_text)

I don't have access to real-time data, so I can't provide the current stock price of Apple. For the most up-to-date information, please check a reliable financial news website or stock market app.


In [4]:
response.__dict__

{'id': 'resp_0405a782a6df762300691b45ad0db481a295a27ec64ba001e0',
 'created_at': 1763394989.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-nano-2025-04-14',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_0405a782a6df762300691b45ad78fc81a299b9a37719c6adc5', content=[ResponseOutputText(annotations=[], text="I don't have access to real-time data, so I can't provide the current stock price of Apple. For the most up-to-date information, please check a reliable financial news website or stock market app.", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'background': False,
 'conversation': None,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'previous_response_id': None,
 'prompt': None,
 'prompt_cache_key': None,
 'reasoning': Reasoning(effort=None, generate_su

In [5]:
response.output[0].__dict__

{'id': 'msg_0405a782a6df762300691b45ad78fc81a299b9a37719c6adc5',
 'content': [ResponseOutputText(annotations=[], text="I don't have access to real-time data, so I can't provide the current stock price of Apple. For the most up-to-date information, please check a reliable financial news website or stock market app.", type='output_text', logprobs=[])],
 'role': 'assistant',
 'status': 'completed',
 'type': 'message'}

## Step 4: Define the input schema for our tool

In [6]:
tools = [{
    "type": "function",
    "name": "get_stock_price",
    "description": "Get the most recent market price of a stock.",
    "parameters": {
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol."
            }
        },
        "required": ["ticker"],
        "additionalProperties": False
    },
    "strict": True
}]

## Step 5: Pass the tool schema over to the model

In [7]:
input_messages = [
    {"role": "user", "content": "What is the real-time stock price of Apple?"}
]

In [8]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools
)

In [9]:
print(response.output_text)

In [10]:
response.__dict__

{'id': 'resp_0527352969cb2cdf00691b45ae2bac81a3935126210fa69242',
 'created_at': 1763394990.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-nano-2025-04-14',
 'object': 'response',
 'output': [ResponseFunctionToolCall(arguments='{"ticker":"AAPL"}', call_id='call_ynUbvJOhkkBY1oqUuntKwKcY', name='get_stock_price', type='function_call', id='fc_0527352969cb2cdf00691b45af16cc81a3815b5b7b339d98e8', status='completed')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [FunctionTool(name='get_stock_price', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string', 'description': 'Stock ticker symbol.'}}, 'required': ['ticker'], 'additionalProperties': False}, strict=True, type='function', description='Get the most recent market price of a stock.')],
 'top_p': 1.0,
 'background': False,
 'conversation': None,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'previous_response_id':

In [11]:
response.output[0].__dict__

{'arguments': '{"ticker":"AAPL"}',
 'call_id': 'call_ynUbvJOhkkBY1oqUuntKwKcY',
 'name': 'get_stock_price',
 'type': 'function_call',
 'id': 'fc_0527352969cb2cdf00691b45af16cc81a3815b5b7b339d98e8',
 'status': 'completed'}

## Step 6: Format the tool call response from the LLM

In [12]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

In [13]:
print(tool_call)

ResponseFunctionToolCall(arguments='{"ticker":"AAPL"}', call_id='call_ynUbvJOhkkBY1oqUuntKwKcY', name='get_stock_price', type='function_call', id='fc_0527352969cb2cdf00691b45af16cc81a3815b5b7b339d98e8', status='completed')


In [14]:
print(args)

{'ticker': 'AAPL'}


## Step 7: Pass on the tool call arguments to our tool/python function

We now need to pass on the arguments received by the model to our python function or tool

In [15]:
result = get_stock_price(args["ticker"])
result

268.17999267578125

## Step 8: Update the message list

In [16]:
input_messages = [
    {"role": "user", "content": "What is the real-time stock price of Apple?"},
    {
        "role": "assistant",
        "content": [{
            "type": "output_text",
            "text": f"I'll get the stock price using the {tool_call.name} function."
        }]
    },
    {
        "role": "user",
        "content": f"The stock price is: {result}"
    }
]

In [17]:
from pprint import pprint
pprint(input_messages)

[{'content': 'What is the real-time stock price of Apple?', 'role': 'user'},
 {'content': [{'text': "I'll get the stock price using the get_stock_price "
                       'function.',
               'type': 'output_text'}],
  'role': 'assistant'},
 {'content': 'The stock price is: 268.17999267578125', 'role': 'user'}]


## Step 9: Pass the message list into the model

In [18]:
response_2 = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools
)

In [19]:
print(response_2.output_text)

The current real-time stock price of Apple is approximately $268.18.


In [20]:
response_2.__dict__

{'id': 'resp_09950fbbbdfaf62f00691b45b07c98819282f07da5b132b27d',
 'created_at': 1763394992.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-nano-2025-04-14',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_09950fbbbdfaf62f00691b45b1470481929aeb9e23b2633d13', content=[ResponseOutputText(annotations=[], text='The current real-time stock price of Apple is approximately $268.18.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [FunctionTool(name='get_stock_price', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string', 'description': 'Stock ticker symbol.'}}, 'required': ['ticker'], 'additionalProperties': False}, strict=True, type='function', description='Get the most recent market price of a stock.')],
 'top_p': 1.0,
 'background': False,
 'conversation': None,
 'max_o